In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install roboflow

In [3]:
import math
import numpy as np
from roboflow import Roboflow

In [4]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="OFJsbzSXtei8j554tCdF")
# project = rf.workspace().project("moving-pingpong-detector")
# model = project.version("1").model
# from PIL import Image

# im = Image.open(r"/content/right_bottom.png")
# im.save(r"/content/pp_test.jpg")
# # infer on a local image
# print(model.predict("/content/pp_test.jpg", confidence=40, overlap=30).json())

# visualize your prediction
# model.predict("your_image.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

In [5]:
rf = Roboflow(api_key="OFJsbzSXtei8j554tCdF")
project = rf.workspace().project("moving-pingpong-detector")
model = project.version("1").model

job_id, signed_url, expire_time = model.predict_video(
    "/content/drive/MyDrive/人工智慧實務/pp_videos/v3.1.mp4",
    fps=29,
    prediction_type="batch-video",
)

results = model.poll_until_video_results(job_id)

loading Roboflow workspace...
loading Roboflow project...
Checking for video inference results for job 85abd0be-facb-463d-bda4-d19bd7cc943a every 60s
(0s): Checking for inference results
(60s): Checking for inference results


In [6]:
def get_move_unmove_balls(all_balls,frame_balls,frame,count):
    move_balls,unmove_balls=[],[]
    for current_ball in frame_balls:
        for previous_ball in all_balls[frame-1]:
            if abs(current_ball['x'] - previous_ball['x'])<10 and abs(current_ball['y'] - previous_ball['y'])<10:
                unmove_balls.append(current_ball)
        if current_ball in unmove_balls:
            pass
        else:
            current_ball['id']=str(count)
            count+=1
            move_balls.append(current_ball)

    return move_balls,unmove_balls,count

def get_speed(x1,y1,x2,y2,h=0.0002775,x_sub=0.306,y_sub=0.2796,time=0.03333):
    #print('getting speed')
    x_diff=((y2+y1)/2-214)*h
    x_sub = x_sub - x_diff
    x_cm=x_sub*(x2-x1)
    y_cm=y_sub*(y2-y1)
    distance=math.sqrt(x_cm**2 + y_cm**2)
    speed = distance/time
    speed_n= speed/100000*3600
    return speed_n

def get_possible_same_balls(move_balls,all_move_balls,frame):
    possible_same_balls = []
    possible_same_balls_vector = []
    for current_ball in move_balls:

        for previous_ball in all_move_balls[frame - 1]:
            # if abs(current_ball['x'] - previous_ball['x']) < 100 and abs(current_ball['y'] - previous_ball['y']) > 50:
                current_ball['id']+= "<-"+previous_ball['id']
                # if previous_ball['x']>=166 and previous_ball['x']<=1066 and previous_ball['y']>=214 and previous_ball['y']<=704:
                current_ball['speed'] = get_speed(previous_ball['x'],previous_ball['y'],current_ball['x'],current_ball['y'])
                possible_same_balls.append(current_ball.copy())
                possible_same_balls_vector.append([current_ball['x'] - previous_ball['x'], current_ball['y'] - previous_ball['y']])
                tmp=current_ball['id'].find('<-')
                tmp_str = current_ball['id'][0:tmp]
                current_ball['id']=tmp_str
    #print(possible_same_balls)
    return possible_same_balls, possible_same_balls_vector

all_balls = []
all_move_balls = []
count=0
for frame in range(len(results['frame_offset'])):
    frame_balls = []

    for ball in results['moving-pingpong-detector'][frame]['predictions'].copy():
        frame_balls.append(ball) #將現在所有偵測到的球加入frame_balls
    all_balls.append(frame_balls) #將每一個frame加入all_balls

    if len(all_balls) > 0:
        move_balls,unmove_balls,count = get_move_unmove_balls(all_balls,frame_balls,frame,count)
        all_move_balls.append(move_balls)

frame=0
for move_balls in all_move_balls:
    print(f'frame:{frame}')
    frame_balls = [[move_balls[i]["id"], math.inf, [0, 0], 0, [0, 0]] for i in range(len(move_balls))]

    if len(move_balls)>0:
        possible_same_balls, possible_same_balls_vector = get_possible_same_balls(move_balls,all_move_balls,frame)
        for i in range(len(possible_same_balls)):
            id = possible_same_balls[i]["id"]
            # 參考: https://stackoverflow.com/questions/17332759/finding-vectors-with-2-points
            norm = math.sqrt(possible_same_balls_vector[i][0] ** 2 + possible_same_balls_vector[i][1] ** 2)
            direction = [possible_same_balls_vector[i][0] / norm, possible_same_balls_vector[i][1] / norm]
            speed = possible_same_balls[i]["speed"]
            position = [possible_same_balls[i]["x"], possible_same_balls[i]["y"]]

            for j in range(len(frame_balls)):
              if id.split("<-")[0] == frame_balls[j][0].split("<-")[0] and norm < frame_balls[j][1]:
                frame_balls[j][0] = id
                frame_balls[j][1] = norm
                frame_balls[j][2] = direction
                frame_balls[j][3] = speed
                frame_balls[j][4] = position

        sub_frame_balls = {}

        for j in range(len(frame_balls)):
          sub_frame_balls[frame_balls[j][0].split("<-")[-1]] = []

        for j in range(len(frame_balls)):
          sub_frame_balls[frame_balls[j][0].split("<-")[-1]].append(frame_balls[j])

        print_balls = []

        for source_ball, balls in sub_frame_balls.items():
          temp_ball = [str(source_ball), math.inf, [0, 0], 0, [0, 0]]
          for ball in balls:
            if ball[1] < temp_ball[1]:
              temp_ball = ball
          temp_ball[1] = "norm: {:.2f}".format(temp_ball[1])
          temp_ball[2] = "dir: " + "[{:.2f}, ".format(temp_ball[2][0]) + "{:.2f}]".format(temp_ball[2][1])
          temp_ball[3] = "speed: {:.2f} km/h".format(temp_ball[3])
          temp_ball[4] = "pos: [{:.2f}, ".format(temp_ball[4][0]) + "{:.2f}]".format(temp_ball[4][1])
          print_balls.append(temp_ball)

        print(print_balls)

    frame+=1
    # possible_same_balls = get_possible_same_balls(move_balls,all_move_balls,frame)


frame:0
frame:1
[['0', 'norm: inf', 'dir: [0.00, 0.00]', 'speed: 0.00 km/h', 'pos: [0.00, 0.00]']]
frame:2
[['1<-0', 'norm: 59.79', 'dir: [0.49, 0.87]', 'speed: 19.12 km/h', 'pos: [584.50, 108.50]']]
frame:3
[['2<-1', 'norm: 76.32', 'dir: [0.37, 0.93]', 'speed: 23.59 km/h', 'pos: [612.50, 179.50]']]
frame:4
[['4<-2', 'norm: 83.72', 'dir: [0.41, 0.91]', 'speed: 25.67 km/h', 'pos: [646.50, 256.00]']]
frame:5
[['5<-4', 'norm: 96.01', 'dir: [0.37, 0.93]', 'speed: 29.03 km/h', 'pos: [682.50, 345.00]']]
frame:6
[['6<-5', 'norm: 109.12', 'dir: [0.39, 0.92]', 'speed: 32.54 km/h', 'pos: [725.00, 445.50]']]
frame:7
[['8<-6', 'norm: 81.61', 'dir: [0.59, 0.81]', 'speed: 23.30 km/h', 'pos: [773.00, 511.50]'], ['9<-7', 'norm: 69.16', 'dir: [-0.14, -0.99]', 'speed: 20.79 km/h', 'pos: [578.50, 572.00]']]
frame:8
[['12<-8', 'norm: 92.96', 'dir: [0.65, 0.76]', 'speed: 25.52 km/h', 'pos: [833.00, 582.50]'], ['13<-9', 'norm: 73.19', 'dir: [-0.14, -0.99]', 'speed: 22.02 km/h', 'pos: [568.50, 499.50]']]
fra